In [4]:
import pandas as pd

In [5]:
df_cards = pd.read_csv('cards_data.csv')
df_users = pd.read_csv('users_data.csv')
df_transactions = pd.read_csv('transactions_data.csv')
df_transactions_city = df_transactions.copy()

# Análise Exploratória:

In [6]:
def mostrar_nulls(dataset):
    for row in dataset:
        print(f'{row}: ', dataset[row].isnull().sum())

In [7]:
print(df_cards.duplicated().sum())
print(df_transactions.duplicated().sum())
print(df_users.duplicated().sum())

0
0
0


## Cards

In [8]:
print('Cards: \n')
df_cards.loc[df_cards['id'] == 4524]

Cards: 



,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No


In [9]:
mostrar_nulls(df_cards)

id:  0
client_id:  0
card_brand:  0
card_type:  0
card_number:  0
expires:  0
cvv:  0
has_chip:  0
num_cards_issued:  0
credit_limit:  0
acct_open_date:  0
year_pin_last_changed:  0
card_on_dark_web:  0


## Users

In [10]:
print('users: \n')
df_users.head()

users: 



,id,current_age,retirement_age,birth_year,birth_month,gender,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards
0,825,53,66,1966,11,Female,462 Rose Lane,34.15,-117.76,$29278,$59696,$127613,787,5
1,1746,53,68,1966,12,Female,3606 Federal Boulevard,40.76,-73.74,$37891,$77254,$191349,701,5
2,1718,81,67,1938,11,Female,766 Third Drive,34.02,-117.89,$22681,$33483,$196,698,5
3,708,63,63,1957,1,Female,3 Madison Street,40.71,-73.99,$163145,$249925,$202328,722,4
4,1164,43,70,1976,9,Male,9620 Valley Stream Drive,37.76,-122.44,$53797,$109687,$183855,675,1


In [11]:
mostrar_nulls(df_users)

id:  0
current_age:  0
retirement_age:  0
birth_year:  0
birth_month:  0
gender:  0
address:  0
latitude:  0
longitude:  0
per_capita_income:  0
yearly_income:  0
total_debt:  0
credit_score:  0
num_credit_cards:  0


## Transactions

In [12]:
print('transactions: \n')
df_transactions.head()

transactions: 



,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN


In [13]:
mostrar_nulls(df_transactions)

id:  0
date:  0
client_id:  0
card_id:  0
amount:  0
use_chip:  0
merchant_id:  0
merchant_city:  0
merchant_state:  1563700
zip:  1652706
mcc:  0
errors:  13094522


In [14]:
df_transactions.shape

(13305915, 12)

# Tratamento de Dados

## Tratando Valores Nulos

In [15]:
df_transactions_temp = df_transactions.head(100000).copy()

In [16]:
df_transactions_temp['amount'] = df_transactions_temp['amount'].str.replace('$', '', regex=False).astype(float)

In [17]:
df_transactions_temp.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN


In [18]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for column in ['use_chip', 'merchant_city', 'merchant_state']:
    label_encoders[column] = LabelEncoder()
    df_transactions_temp[column] = label_encoders[column].fit_transform(df_transactions_temp[column])

In [19]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
import pandas as pd

label_encoders = {}

for column in ['use_chip', 'merchant_city', 'merchant_state']:
    label_encoders[column] = LabelEncoder()
    df_transactions_temp[column] = label_encoders[column].fit_transform(df_transactions_temp[column])

knn_imputer = KNNImputer(n_neighbors=5)

numerical_columns = df_transactions_temp.select_dtypes(include=['float64', 'int64']).columns
non_numerical_columns = df_transactions_temp.select_dtypes(exclude=['float64', 'int64']).columns

df_numerical_imputed = pd.DataFrame(
    knn_imputer.fit_transform(df_transactions_temp[numerical_columns]),
    columns=numerical_columns,
    index=df_transactions_temp.index
)

df_transactions_imputed = pd.concat([df_numerical_imputed, df_transactions_temp[non_numerical_columns]], axis=1)

In [20]:
for column in ['use_chip', 'merchant_city', 'merchant_state']:
    df_transactions_imputed[column] = df_transactions_imputed[column].fillna(-1).astype(int)
    df_transactions_imputed[column] = label_encoders[column].inverse_transform(df_transactions_imputed[column])

In [21]:
#del df_transactions_temp
#del df_transaction

In [22]:
mostrar_nulls(df_transactions_imputed)

id:  0
client_id:  0
card_id:  0
amount:  0
use_chip:  0
merchant_id:  0
merchant_city:  0
merchant_state:  0
zip:  0
mcc:  0
date:  0
errors:  98448


In [23]:
df_transactions_imputed['merchant_city'] = df_transactions_city['merchant_city']
df_transactions_imputed['merchant_state'] = df_transactions['merchant_state']
df_transactions_imputed['merchant_id'] = df_transactions_city['merchant_id']
df_transactions_imputed['id'] = df_transactions_imputed['id'].astype('Int64')
df_transactions_imputed['client_id'] = df_transactions_imputed['client_id'].astype('Int64')
df_transactions_imputed['card_id'] = df_transactions_imputed['card_id'].astype('Int64')
df_transactions_imputed['zip'] = df_transactions_imputed['zip'].round().astype(int)
df_transactions_imputed['zip'] = df_transactions_imputed['zip'].astype('Int64')
df_transactions_imputed['mcc'] = df_transactions_imputed['mcc'].astype('Int64')
df_transactions_imputed['use_chip'] = df_transactions_imputed['use_chip'].astype('boolean')

df_transactions_imputed.head()

,id,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,date,errors
0,7475327,1556,2972,-77.00,True,59935,Beulah,ND,58523,5499,2010-01-01 00:01:00,NaN
1,7475328,561,4575,14.57,True,67570,Bettendorf,IA,52722,5311,2010-01-01 00:02:00,NaN
2,7475329,1129,102,80.00,True,27092,Vista,CA,92084,4829,2010-01-01 00:02:00,NaN
3,7475331,430,2860,200.00,True,27092,Crown Point,IN,46307,4829,2010-01-01 00:05:00,NaN
4,7475332,848,3915,46.41,True,13051,Harwood,MD,20776,5813,2010-01-01 00:06:00,NaN


# O que fazer
- Transformar as tabelas de acordo com o diagrama
- Tratar os dados
- Dashboard (Final)

In [24]:
'''
for row in df_users['address']:   nao funciona pq row não tá sendo tratado como o index, mas sim o valor da linha
    df_users['address'][row] = df_users['address'][row].split(' ', 1)
    df_users['number'] = df_users['address'][row][0]
    df_users['address'][row].pop()
'''

"\nfor row in df_users['address']:   nao funciona pq row não tá sendo tratado como o index, mas sim o valor da linha\n    df_users['address'][row] = df_users['address'][row].split(' ', 1)\n    df_users['number'] = df_users['address'][row][0]\n    df_users['address'][row].pop()\n"

In [25]:
df_users[['number', 'street']] = df_users['address'].str.split(' ', n=1, expand=True)
df_users = df_users.drop('address', axis=1)

df_users

,id,current_age,retirement_age,birth_year,birth_month,gender,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,number,street
0,825,53,66,1966,11,Female,34.15,-117.76,$29278,$59696,$127613,787,5,462,Rose Lane
1,1746,53,68,1966,12,Female,40.76,-73.74,$37891,$77254,$191349,701,5,3606,Federal Boulevard
2,1718,81,67,1938,11,Female,34.02,-117.89,$22681,$33483,$196,698,5,766,Third Drive
3,708,63,63,1957,1,Female,40.71,-73.99,$163145,$249925,$202328,722,4,3,Madison Street
4,1164,43,70,1976,9,Male,37.76,-122.44,$53797,$109687,$183855,675,1,9620,Valley Stream Drive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,986,32,70,1987,7,Male,40.65,-73.58,$23550,$48010,$87837,703,3,6577,Lexington Lane
1996,1944,62,65,1957,11,Female,38.95,-84.54,$24218,$49378,$104480,740,4,2,Elm Drive
1997,185,47,67,1973,1,Female,40.66,-74.19,$15175,$30942,$71066,779,3,276,Fifth Boulevard
1998,1007,66,60,1954,2,Male,40.24,-76.92,$25336,$54654,$27241,618,1,259,Valley Boulevard


In [40]:
df_merchants = df_transactions_imputed[['merchant_id', 'merchant_city', 'merchant_state']]

df_transactions_errors = df_transactions_imputed['errors']

df_adresses = df_users[['number', 'street','latitude','longitude','id']]
df_user_adress = df_users[['id']]
df_users
df_transactions_imputed.dtypes

id                  Int64
client_id           Int64
card_id             Int64
amount            float64
use_chip          boolean
merchant_id         int64
merchant_city      object
merchant_state     object
zip                 Int64
mcc                 Int64
date               object
errors             object
dtype: object

In [41]:
df_cards.head()

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


In [27]:
df_users

,id,current_age,retirement_age,birth_year,birth_month,gender,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,number,street
0,825,53,66,1966,11,Female,34.15,-117.76,$29278,$59696,$127613,787,5,462,Rose Lane
1,1746,53,68,1966,12,Female,40.76,-73.74,$37891,$77254,$191349,701,5,3606,Federal Boulevard
2,1718,81,67,1938,11,Female,34.02,-117.89,$22681,$33483,$196,698,5,766,Third Drive
3,708,63,63,1957,1,Female,40.71,-73.99,$163145,$249925,$202328,722,4,3,Madison Street
4,1164,43,70,1976,9,Male,37.76,-122.44,$53797,$109687,$183855,675,1,9620,Valley Stream Drive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,986,32,70,1987,7,Male,40.65,-73.58,$23550,$48010,$87837,703,3,6577,Lexington Lane
1996,1944,62,65,1957,11,Female,38.95,-84.54,$24218,$49378,$104480,740,4,2,Elm Drive
1997,185,47,67,1973,1,Female,40.66,-74.19,$15175,$30942,$71066,779,3,276,Fifth Boulevard
1998,1007,66,60,1954,2,Male,40.24,-76.92,$25336,$54654,$27241,618,1,259,Valley Boulevard


In [28]:
df_merchants

,merchant_id,merchant_city,merchant_state
0,59935,Beulah,ND
1,67570,Bettendorf,IA
2,27092,Vista,CA
3,27092,Crown Point,IN
4,13051,Harwood,MD
...,...,...,...
99995,54850,Panama City,FL
99996,83480,Alcoa,TN
99997,36934,Houston,TX
99998,39021,ONLINE,NaN


In [29]:
df_adresses

,number,street,latitude,longitude,id
0,462,Rose Lane,34.15,-117.76,825
1,3606,Federal Boulevard,40.76,-73.74,1746
2,766,Third Drive,34.02,-117.89,1718
3,3,Madison Street,40.71,-73.99,708
4,9620,Valley Stream Drive,37.76,-122.44,1164
...,...,...,...,...,...
1995,6577,Lexington Lane,40.65,-73.58,986
1996,2,Elm Drive,38.95,-84.54,1944
1997,276,Fifth Boulevard,40.66,-74.19,185
1998,259,Valley Boulevard,40.24,-76.92,1007


In [30]:
df_transactions.dtypes

id                  int64
date               object
client_id           int64
card_id             int64
amount             object
use_chip           object
merchant_id         int64
merchant_city      object
merchant_state     object
zip               float64
mcc                 int64
errors             object
dtype: object

In [31]:
df_merchants.dtypes

merchant_id        int64
merchant_city     object
merchant_state    object
dtype: object

In [32]:
print(df_cards.duplicated().sum())
print(df_transactions.duplicated().sum())
print(df_users.duplicated().sum())

0
0
0


In [33]:
df_completo = pd.concat([df_adresses, df_users, df_user_adress, df_cards, df_merchants, df_transactions_imputed, df_transactions_errors])

In [34]:
df_completo

,number,street,latitude,longitude,id,current_age,retirement_age,birth_year,birth_month,gender,...,merchant_id,merchant_city,merchant_state,card_id,amount,use_chip,zip,mcc,date,errors
0,462,Rose Lane,34.15,-117.76,825,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN
1,3606,Federal Boulevard,40.76,-73.74,1746,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN
2,766,Third Drive,34.02,-117.89,1718,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN
3,3,Madison Street,40.71,-73.99,708,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN
4,9620,Valley Stream Drive,37.76,-122.44,1164,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN
99996,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN
99997,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN
99998,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,<NA>,NaN,<NA>,<NA>,<NA>,NaN,NaN


In [35]:
columns_to_clean = ["per_capita_income", "yearly_income", "total_debt"]
for column in columns_to_clean:
    df_users[column] = df_users[column].str.replace("$", "").astype(float)


In [36]:
df_users.to_csv('df_users.csv')
df_transactions_imputed.to_csv('df_transactions_imputed.csv')
df_cards.to_csv('df_cards.csv')
df_adresses.to_csv('df_adresses.csv')

In [37]:
df_users

,id,current_age,retirement_age,birth_year,birth_month,gender,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,number,street
0,825,53,66,1966,11,Female,34.15,-117.76,29278.0,59696.0,127613.0,787,5,462,Rose Lane
1,1746,53,68,1966,12,Female,40.76,-73.74,37891.0,77254.0,191349.0,701,5,3606,Federal Boulevard
2,1718,81,67,1938,11,Female,34.02,-117.89,22681.0,33483.0,196.0,698,5,766,Third Drive
3,708,63,63,1957,1,Female,40.71,-73.99,163145.0,249925.0,202328.0,722,4,3,Madison Street
4,1164,43,70,1976,9,Male,37.76,-122.44,53797.0,109687.0,183855.0,675,1,9620,Valley Stream Drive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,986,32,70,1987,7,Male,40.65,-73.58,23550.0,48010.0,87837.0,703,3,6577,Lexington Lane
1996,1944,62,65,1957,11,Female,38.95,-84.54,24218.0,49378.0,104480.0,740,4,2,Elm Drive
1997,185,47,67,1973,1,Female,40.66,-74.19,15175.0,30942.0,71066.0,779,3,276,Fifth Boulevard
1998,1007,66,60,1954,2,Male,40.24,-76.92,25336.0,54654.0,27241.0,618,1,259,Valley Boulevard


In [38]:
df_users.dtypes

id                     int64
current_age            int64
retirement_age         int64
birth_year             int64
birth_month            int64
gender                object
latitude             float64
longitude            float64
per_capita_income    float64
yearly_income        float64
total_debt           float64
credit_score           int64
num_credit_cards       int64
number                object
street                object
dtype: object

In [39]:
df_transactions_imputed.head()

,id,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,date,errors
0,7475327,1556,2972,-77.00,True,59935,Beulah,ND,58523,5499,2010-01-01 00:01:00,NaN
1,7475328,561,4575,14.57,True,67570,Bettendorf,IA,52722,5311,2010-01-01 00:02:00,NaN
2,7475329,1129,102,80.00,True,27092,Vista,CA,92084,4829,2010-01-01 00:02:00,NaN
3,7475331,430,2860,200.00,True,27092,Crown Point,IN,46307,4829,2010-01-01 00:05:00,NaN
4,7475332,848,3915,46.41,True,13051,Harwood,MD,20776,5813,2010-01-01 00:06:00,NaN
